# Finalizing a Prediction Strategy

In previous notebooks, we explored different data preprocessing techniques, feature engineering, and testing various sklearn models for predicting NBA games.

Now, we will attempt to finalize a prediction strategy based on what we've learned.

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

## Read the dataset

In [2]:
path_to_data = "../../data/processed/processed_team_dataset.csv"

df = pd.read_csv(path_to_data, index_col=0)
df

,date,season,team,team_opp,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,2015-10-29,2016,NYK,ATL,1356.009483,1541.442237,0.379472,-3.051170,1,0,...,41.20,24.80,35.60,3.20,4.70,33.30,23.60,132.0,104.0,106.0
1,2015-10-29,2016,ATL,NYK,1541.442237,1356.009483,0.620528,3.051170,0,1,...,20.50,20.30,34.90,3.40,6.80,50.00,32.50,140.0,137.0,97.0
2,2015-10-29,2016,LAC,DAL,1652.435518,1559.753058,0.751974,6.881516,1,0,...,38.50,28.10,32.20,4.20,4.40,66.70,27.90,162.0,114.0,95.0
3,2015-10-29,2016,IND,MEM,1499.317744,1557.619147,0.559723,1.489236,1,0,...,34.50,23.50,30.60,8.40,12.20,100.00,28.00,149.0,89.0,106.0
4,2015-10-29,2016,MEM,IND,1557.619147,1499.317744,0.440277,-1.489236,0,1,...,42.50,24.80,30.20,2.60,4.80,100.00,29.70,134.0,105.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,2024-02-15,2024,MIL,MEM,1539.547927,1410.179450,0.542164,1.048874,0,1,...,35.17,26.01,46.02,5.42,9.31,36.66,30.13,176.2,115.0,114.4
21752,2024-02-15,2024,MEM,MIL,1410.179450,1539.547927,0.457836,-1.048874,1,0,...,40.78,23.44,43.28,5.62,7.04,37.04,43.73,175.3,122.7,113.4
21753,2024-02-15,2024,MIN,POR,1676.024592,1333.586609,0.801482,8.658499,0,1,...,33.23,22.67,39.22,13.52,8.59,36.08,38.80,180.1,124.0,114.5
21754,2024-02-15,2024,POR,MIN,1333.586609,1676.024592,0.198518,-8.658499,1,0,...,38.21,23.52,44.58,4.41,9.39,45.89,35.88,191.6,129.7,103.6


In [3]:
# Check that we have no null values

# Finding the number of missing values in each column
missing_values = df.isnull().sum()

# Printing the number of missing values in each column
print(sum(missing_values))

0


## Separate relevant features from the labels

In [4]:
# drop these columns (most of the are non_numeric or are not useful for machine learning)
drop_columns = ["date", "season", "team", "team_opp", "won"]

selected_columns = df.columns[~df.columns.isin(drop_columns)]

# Selected features
features_df = df[selected_columns]
features_df

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,1356.009483,1541.442237,0.379472,-3.051170,1,0,42.0,93.0,0.4520,9.0,...,41.20,24.80,35.60,3.20,4.70,33.30,23.60,132.0,104.0,106.0
1,1541.442237,1356.009483,0.620528,3.051170,0,1,37.0,82.0,0.4510,8.0,...,20.50,20.30,34.90,3.40,6.80,50.00,32.50,140.0,137.0,97.0
2,1652.435518,1559.753058,0.751974,6.881516,1,0,42.0,80.0,0.5250,6.0,...,38.50,28.10,32.20,4.20,4.40,66.70,27.90,162.0,114.0,95.0
3,1499.317744,1557.619147,0.559723,1.489236,1,0,32.0,86.0,0.3720,9.0,...,34.50,23.50,30.60,8.40,12.20,100.00,28.00,149.0,89.0,106.0
4,1557.619147,1499.317744,0.440277,-1.489236,0,1,29.0,82.0,0.3540,2.0,...,42.50,24.80,30.20,2.60,4.80,100.00,29.70,134.0,105.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,1539.547927,1410.179450,0.542164,1.048874,0,1,42.0,87.1,0.4819,15.0,...,35.17,26.01,46.02,5.42,9.31,36.66,30.13,176.2,115.0,114.4
21752,1410.179450,1539.547927,0.457836,-1.048874,1,0,37.4,85.4,0.4387,14.3,...,40.78,23.44,43.28,5.62,7.04,37.04,43.73,175.3,122.7,113.4
21753,1676.024592,1333.586609,0.801482,8.658499,0,1,42.0,83.8,0.5023,13.9,...,33.23,22.67,39.22,13.52,8.59,36.08,38.80,180.1,124.0,114.5
21754,1333.586609,1676.024592,0.198518,-8.658499,1,0,40.4,86.0,0.4683,10.4,...,38.21,23.52,44.58,4.41,9.39,45.89,35.88,191.6,129.7,103.6


In [5]:
# Label we want to predict
label = df["won"]
label

0        False
1         True
2         True
3        False
4         True
         ...  
21751    False
21752     True
21753     True
21754    False
21755    False
Name: won, Length: 21756, dtype: bool

### Remember that elo is about 64.48% accurate at predicting winner of a game

In [6]:
projected_point_spread = df["team_point_diff_proj"]
projected_win_from_elo = projected_point_spread > 0

print(metrics.accuracy_score(label, projected_win_from_elo))

0.6485567199852914


## Scale data for Logistic Regression

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_features_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)
scaled_features_df

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,0.240976,0.517230,0.374891,0.438248,1.0,0.0,0.548387,0.454545,0.562044,0.391304,...,0.264080,0.135632,0.182741,0.062500,0.105856,0.218970,0.000000,0.120419,0.303571,0.447761
1,0.517230,0.240976,0.625109,0.561752,0.0,1.0,0.387097,0.204545,0.558394,0.347826,...,0.005006,0.083908,0.173858,0.071429,0.153153,0.414520,0.116492,0.162304,0.892857,0.313433
2,0.682585,0.544509,0.761551,0.639273,1.0,0.0,0.548387,0.159091,0.828467,0.260870,...,0.230288,0.173563,0.139594,0.107143,0.099099,0.610070,0.056283,0.277487,0.482143,0.283582
3,0.454473,0.541330,0.561992,0.530140,1.0,0.0,0.225806,0.295455,0.270073,0.391304,...,0.180225,0.120690,0.119289,0.294643,0.274775,1.000000,0.057592,0.209424,0.035714,0.447761
4,0.541330,0.454473,0.438008,0.469860,0.0,1.0,0.129032,0.204545,0.204380,0.086957,...,0.280350,0.135632,0.114213,0.035714,0.108108,1.000000,0.079843,0.130890,0.321429,0.447761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,0.514407,0.321677,0.543767,0.521228,0.0,1.0,0.548387,0.320455,0.671168,0.652174,...,0.188611,0.149540,0.314975,0.161607,0.209685,0.258314,0.085471,0.351832,0.500000,0.573134
21752,0.321677,0.514407,0.456233,0.478772,1.0,0.0,0.400000,0.281818,0.513504,0.621739,...,0.258824,0.120000,0.280203,0.170536,0.158559,0.262763,0.263482,0.347120,0.637500,0.558209
21753,0.717727,0.207571,0.812940,0.675237,0.0,1.0,0.548387,0.245455,0.745620,0.604348,...,0.164330,0.111149,0.228680,0.523214,0.193468,0.251522,0.198953,0.372251,0.660714,0.574627
21754,0.207571,0.717727,0.187060,0.324763,1.0,0.0,0.496774,0.295455,0.621533,0.452174,...,0.226658,0.120920,0.296701,0.116518,0.211486,0.366393,0.160733,0.432461,0.762500,0.411940


## Test Train Split

In [8]:
from sklearn.model_selection import train_test_split

test_ratio = 0.3
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(scaled_features_df, label, test_size = 0.3) # 70% data is training and 30% is for testing

x_train_scaled

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
6770,0.344766,0.693856,0.113196,0.258347,0.0,1.0,0.477419,0.313636,0.591971,0.347826,...,0.178348,0.103563,0.273223,0.266518,0.170045,0.302576,0.115838,0.389005,0.616071,0.368657
9762,0.550270,0.625349,0.573600,0.535854,1.0,0.0,0.541935,0.352273,0.641971,0.460870,...,0.225031,0.113563,0.297335,0.129018,0.163288,0.281967,0.134686,0.279581,0.712500,0.535821
13308,0.243502,0.576397,0.120585,0.266204,0.0,1.0,0.454839,0.309091,0.559124,0.339130,...,0.247559,0.131954,0.382234,0.208482,0.192117,0.346604,0.153272,0.491623,0.694643,0.549254
4022,0.373242,0.617642,0.167279,0.309140,0.0,1.0,0.425806,0.143182,0.675182,0.430435,...,0.377096,0.194713,0.223477,0.246429,0.270045,0.251991,0.175654,0.325131,0.603571,0.549254
17906,0.601174,0.528195,0.709850,0.607689,1.0,0.0,0.496774,0.463636,0.495620,0.582609,...,0.212766,0.127011,0.271954,0.161161,0.190991,0.241920,0.211911,0.410995,0.626786,0.576119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16935,0.570363,0.338826,0.582088,0.540056,0.0,1.0,0.509677,0.429545,0.531752,0.447826,...,0.171589,0.113563,0.212817,0.232143,0.195495,0.192623,0.146466,0.513613,0.589286,0.643284
20783,0.148399,0.435230,0.143520,0.288554,0.0,1.0,0.493548,0.397727,0.541606,0.591304,...,0.149061,0.084138,0.400000,0.160268,0.229730,0.287002,0.148822,0.350785,0.632143,0.474627
4453,0.583246,0.552094,0.383854,0.442833,0.0,1.0,0.380645,0.172727,0.588686,0.547826,...,0.231539,0.127126,0.310660,0.127679,0.177477,0.243794,0.092670,0.258639,0.542857,0.428358
15767,0.385916,0.738466,0.111661,0.256668,0.0,1.0,0.493548,0.288636,0.627737,0.378261,...,0.208385,0.093333,0.366878,0.181696,0.184685,0.296136,0.207853,0.443979,0.835714,0.473134


In [9]:
from sklearn.linear_model import LogisticRegression 

# Logistic Regression with max_iter=200 
log_model = LogisticRegression(max_iter=200, verbose=2, random_state=42)
log_model.fit(x_train_scaled, y_train_scaled)
y_pred_log = log_model.predict(x_test_scaled)
print(metrics.accuracy_score(y_test_scaled, y_pred_log))

0.654512026964915


From our previous tests, we found that logistic regression gave around 65% accuracy. Let us try to find the optimal parameters.

In [10]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'C': [0.001, 0.01, 0.1, 0.5, 0.75, 1, 1.25, 1.5, 2, 5, 10, 100],
#     'penalty': ['l1', 'l2', 'elasticnet', 'none'],
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# }

# log_model = LogisticRegression(max_iter=1000)
# clf = GridSearchCV(log_model, param_grid, cv=5, scoring='accuracy', verbose=3)

In [11]:
# clf.fit(x_train_scaled, y_train_scaled)

In [12]:
# print("Best Parameters:", clf.best_params_)

### Train using best parameters

We have found "optimal" parameters with GridSearchCV:

Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}

Let's evaluate the model using these parameters.

In [13]:
opt_log_model = LogisticRegression(max_iter=1000, verbose=2, random_state=42, C=0.01, penalty='l2', solver='liblinear')
opt_log_model.fit(x_train_scaled, y_train_scaled)
y_pred = opt_log_model.predict(x_test_scaled)
print(metrics.accuracy_score(y_test_scaled, y_pred))

[LibLinear]0.6546652367090547


It seems like the parameters do not give much of a performance bump. Nonetheless, let's use this "optimized" model further.

### See accuracy of "high" confidence predictions

In [22]:
# Predict with a probability for each class
y_pred_prob_log = opt_log_model.predict_proba(x_test_scaled)
y_pred_prob_log

array([[0.43681317, 0.56318683],
       [0.71498664, 0.28501336],
       [0.73222266, 0.26777734],
       ...,
       [0.55294399, 0.44705601],
       [0.6364238 , 0.3635762 ],
       [0.52168267, 0.47831733]])

In [23]:
prob_threshold = 0.7

# Identify predictions where the probability of either class 0 or class 1 meets or exceeds the threshold
high_confidence_indices = np.where((y_pred_prob_log[:, 0] >= prob_threshold) | (y_pred_prob_log[:, 1] >= prob_threshold))

high_confidence_indices = list(high_confidence_indices[0])
print(len(high_confidence_indices))

1317


In [24]:
# Extract the predictions and actual labels for these high confidence predictions
high_conf_predictions = np.argmax(y_pred_prob_log[high_confidence_indices], axis=1)
high_conf_actual = y_test_scaled.reset_index(drop=True)
high_conf_actual = high_conf_actual[high_confidence_indices]
high_conf_actual

1       False
2        True
6        True
10       True
12       True
        ...  
6503    False
6509    False
6510    False
6517     True
6522     True
Name: won, Length: 1317, dtype: bool

In [25]:
print(metrics.accuracy_score(high_conf_actual, high_conf_predictions))

0.8116932422171602


### See accuracy of "low" confidence predictions

In [26]:
## What about the accuracy of those outside of the "high_conf_predictions"?

low_confidence_indices = [i for i in range(len(y_pred_prob_log)) if i not in high_confidence_indices]
print(len(low_confidence_indices))
assert len(low_confidence_indices) + len(high_confidence_indices) == len(y_pred_prob_log)

5210


In [27]:
# Extract the predictions and actual labels for these "low" confidence predictions
low_conf_predictions = np.argmax(y_pred_prob_log[low_confidence_indices], axis=1)
low_conf_actual = y_test_scaled.reset_index(drop=True)
low_conf_actual = low_conf_actual[low_confidence_indices]
low_conf_actual

0        True
3        True
4        True
5       False
7       False
        ...  
6521    False
6523    False
6524     True
6525    False
6526    False
Name: won, Length: 5210, dtype: bool

In [28]:
print(metrics.accuracy_score(low_conf_actual, low_conf_predictions))

0.6149712092130518


In [29]:
x_test_low_confidence = x_test_scaled.reset_index(drop=True)
x_test_low_confidence = x_test_low_confidence.loc[low_confidence_indices]
assert(x_test_low_confidence.index.values.tolist() == low_conf_actual.index.values.tolist())
x_test_low_confidence

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,0.611901,0.438706,0.524265,0.511750,0.0,1.0,0.490323,0.413636,0.527372,0.673913,...,0.312265,0.148851,0.278553,0.210714,0.138964,0.217564,0.149346,0.403665,0.555357,0.531343
3,0.520991,0.647363,0.522653,0.510968,1.0,0.0,0.623656,0.487374,0.628548,0.449275,...,0.129328,0.074457,0.431613,0.156250,0.155656,0.280380,0.148633,0.410704,0.632937,0.374793
4,0.379573,0.459394,0.568934,0.533554,1.0,0.0,0.374194,0.300000,0.460584,0.508696,...,0.212516,0.150805,0.282995,0.173214,0.245721,0.273302,0.206545,0.449215,0.783929,0.720896
5,0.338949,0.453942,0.534027,0.516489,1.0,0.0,0.677419,0.479545,0.706204,0.547826,...,0.160701,0.092759,0.253553,0.174107,0.163964,0.300234,0.103403,0.281152,0.576786,0.588060
7,0.616407,0.588987,0.380306,0.441022,0.0,1.0,0.461290,0.247727,0.616058,0.582609,...,0.226158,0.164828,0.299492,0.137946,0.206532,0.375995,0.171597,0.437696,0.610714,0.495522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6521,0.575707,0.686969,0.537751,0.518299,1.0,0.0,0.522581,0.384091,0.578467,0.534783,...,0.209512,0.127931,0.219543,0.153125,0.171171,0.215925,0.109555,0.357592,0.605357,0.370149
6523,0.267683,0.705824,0.236874,0.359703,1.0,0.0,0.451613,0.284091,0.582117,0.195652,...,0.194618,0.135057,0.351523,0.276786,0.179054,0.707260,0.171466,0.468586,0.517857,0.679104
6524,0.244748,0.295798,0.308853,0.402950,0.0,1.0,0.529032,0.365909,0.601460,0.373913,...,0.217146,0.127011,0.220558,0.288393,0.225676,0.224005,0.168979,0.446073,0.582143,0.555224
6525,0.234451,0.596070,0.298021,0.396831,1.0,0.0,0.438710,0.272727,0.571168,0.543478,...,0.187109,0.138736,0.256472,0.115179,0.245270,0.483489,0.121859,0.452880,0.605357,0.591045


In [30]:
elo_proj_low_confidence = x_test_low_confidence["team_point_diff_proj"] > 0
elo_proj_low_confidence

0       True
3       True
4       True
5       True
7       True
        ... 
6521    True
6523    True
6524    True
6525    True
6526    True
Name: team_point_diff_proj, Length: 5210, dtype: bool

In [31]:
print(metrics.accuracy_score(low_conf_actual, elo_proj_low_confidence))

0.49558541266794626


## K-fold Cross Validation

Let's see if we can improve upon training our model. What we can try to do is a [k-fold cross validation](https://www.youtube.com/watch?v=kituDjzXwfE&t=698s).

In [ ]:
from sklearn.model_selection import cross_val_score

from sklearn import metrics, cross_validation
logreg=LogisticRegression()
predicted = cross_validation.cross_val_predict(logreg, X, y, cv=10)
print metrics.accuracy_score(y, predicted)
print metrics.classification_report(y, predicted) 